# Phase 1 

### Preprocessing

In [1]:
import pandas as pd
import numpy as np
import os
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer

### Now load the pickle data into dataframes from the Output folder one directory above

In [2]:
def loadEverything():
    all_projects_bugreports = pd.read_pickle('../Output/allBugReports.pickle')
    print("*** All Bug Reports are Loaded. ***")
    all_projects_source_codes = pd.read_pickle('../Output/allSourceCodes.pickle')
    print("*** All Source Codes are Loaded. ***")
    return all_projects_bugreports, all_projects_source_codes

all_projects_bugreports, all_projects_source_codes = loadEverything()
display(all_projects_bugreports.iloc[0])
display(all_projects_bugreports.iloc[0].fix)
display(all_projects_bugreports.iloc[0].summary)
display(all_projects_bugreports.iloc[0].description)
# display(all_projects_source_codes.iloc[0].unprocessed_code)

all_projects_bugreports

*** All Bug Reports are Loaded. ***
*** All Source Codes are Loaded. ***


fix                  [org.apache.commons.collections.map.flat3map.j...
text                                                               NaN
fixdate                                            2006-07-18 22:02:11
summary              Flat3Map.Entry.setValue() overwrites other Ent...
description          Flat3Map&amp;apos;s Entry objects will overwri...
project                                                    COLLECTIONS
average_precision                                                    0
Name: 217, dtype: object

array(['org.apache.commons.collections.map.flat3map.java',
       'org.apache.commons.collections.map.testflat3map.java'],
      dtype='<U52')

'Flat3Map.Entry.setValue() overwrites other Entry values'

'Flat3Map&amp;apos;s Entry objects will overwrite other Entry&amp;apos;s values if Entry.setValue() is called on one.  It should only overwrite the Entry at hand.\nI&amp;apos;ve looked at the source, and the case statement incorrectly falls through, rather than returning like it should:\nFlat3Map.java, lines 646-660:\n        public Object setValue(Object value) {\n            if (canRemove == false) \n{\n                throw new IllegalStateException(AbstractHashedMap.SETVALUE_INVALID);\n            }\n            Object old = getValue();\n            switch (nextIndex) \n{\n                case 3: \n                    parent.value3 = value;\n                case 2:\n                    parent.value2 = value;\n                case 1:\n                    parent.value1 = value;\n            }\n            return old;\n        }\nWith this code, if I set the value of the third item in the EntrySet, then all three values are set to the new value.'

,fix,text,fixdate,summary,description,project,average_precision
id,,,,,,,
217,[org.apache.commons.collections.map.flat3map.j...,NaN,2006-07-18 22:02:11,Flat3Map.Entry.setValue() overwrites other Ent...,Flat3Map&amp;apos;s Entry objects will overwri...,COLLECTIONS,0.0
214,[org.apache.commons.collections.testextendedpr...,NaN,2006-07-18 22:44:33,ExtendedProperties - field include should be n...,"The field ""include"" in ExtendedProperties is c...",COLLECTIONS,0.0
222,[org.apache.commons.collections.testlistutils....,NaN,2006-08-18 19:01:22,CollectionUtils removeAll is actually retainAll,"The removeAll(Collection collection, Collectio...",COLLECTIONS,0.0
261,[org.apache.commons.collections.map.flat3map.j...,NaN,2007-08-20 14:11:54,Flat3Map.remove() does not return the correct ...,final Flat3Map m = new Flat3Map();\n ...,COLLECTIONS,0.0
264,[org.apache.commons.collections.fasttreemap.java],NaN,2007-08-31 09:39:59,FastTreeMap forgets the comparator,In line 359 and 582 of the current 3.2 release...,COLLECTIONS,0.0
...,...,...,...,...,...,...,...
692,[org.wildfly.security.auth.realm.legacypropert...,NaN,2016-11-02 09:35:48,Add tests for special chars in LegacyPropertie...,Add tests for issue https://issues.jboss.org/b...,ELY,0.0
691,[org.wildfly.security.auth.realm.legacypropert...,NaN,2016-11-02 09:36:13,Elytron properties-realm is not compatible wit...,When users properties file (e.g. mgmt-users.pr...,ELY,0.0
637,[org.wildfly.security.auth.server.serverauthen...,NaN,2016-11-03 15:03:29,No log messages comming from Elytron - permiss...,Elytron is missing any log messages related to...,ELY,0.0


In [3]:

display(all_projects_bugreports.iloc[0].fix)
display(all_projects_source_codes.filename)


array(['org.apache.commons.collections.map.flat3map.java',
       'org.apache.commons.collections.map.testflat3map.java'],
      dtype='<U52')

0     \gitrepo\src\java\org\apache\commons\collectio...
1     \gitrepo\src\java\org\apache\commons\collectio...
2     \gitrepo\src\java\org\apache\commons\collectio...
3     \gitrepo\src\java\org\apache\commons\collectio...
4     \gitrepo\src\java\org\apache\commons\collectio...
                            ...                        
63    \gitrepo\src\main\java\org\wildfly\security\ut...
64    \gitrepo\src\main\java\org\wildfly\security\_p...
65    \gitrepo\src\test\java\org\wildfly\security\ma...
66    \gitrepo\src\test\java\org\wildfly\security\ma...
67    \gitrepo\src\test\java\org\wildfly\security\ss...
Name: filename, Length: 10461, dtype: object

## Removing composite varaibles

In [4]:
#remove next line characters:
def remove_new_lines(x):
    return str(x).replace('\n', '').replace('*', '').replace('/', '').replace('\t','')

def clean_new_lines_source_code(df):
    df.unprocessed_code = df.unprocessed_code.apply(remove_new_lines)
    return df

def clean_new_lines_bug_report(df):
    df.summary = df.summary.apply(remove_new_lines)
    df['description'] = df['description'].astype('|S')
    df.description = df.description.apply(remove_new_lines)
    return df

def format_filename(x):
  
    x = x.split('apache')
    if len(x) != 2:
        x = x[0].split('springframework')
        if len(x) != 2:
            x = x[0].split('wildfly')
            if len(x) == 2:
                x = 'org.wildfly' + x[1]
        else:
             x = 'org.springframework' + x[1]
    else:
         x = 'org.apache' + x[1]
    if len(x) == 1:
        x = x[0]
   
    x = str(x).replace("\\",".")
    return x

def clean_filepath(df):
    df.filename = df.filename.apply(format_filename)
    return df



In [5]:
import re

#splitting composite words
def findCompositeWords(s):
    return ' '.join(re.findall('[A-Z][^A-Z]*', s))   


def clean_composite_source_code(df):
    df.unprocessed_code = df.unprocessed_code.apply(findCompositeWords)
    return df

def clean_composite_bug_report(df):
    df.summary = df.summary.apply(findCompositeWords)
    df.description = df.description.apply(findCompositeWords)
    return df


### put Kelvin's key word cleaning above this

In [6]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
# add the Java key words to the stop words

# TODO we need to add the operators in here too
java_keywords = ["abstract", "assert**","assert", "boolean", "break", "byte", "case", "catch", "char", "const", "continue", "default", "do", "double", "else", "enum", "enum****" "extends", "final", "finally", "for", "goto","goto*", "if", "implements", "import", "instanceof", "int","interface", "long", "native", "new", "package", "private", "protected", "public", "return", "short", "static", "strictfp**","strictfp", "super", "switch", "synchornized", "this", "throw", "throws", "transient", "try", "void", "volatile", "while"]
STOP_WORDS = ENGLISH_STOP_WORDS.union(java_keywords)
# STOP_WORDS


#vect = CountVectorizer(min_df = 5, stop_words = stop, analyzer = 'word')

### Start working on TF-IDF and Cosine similarity calculations

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# clean up the unprocessed code column
def clean_source_df(sc_df):
    sc_df = clean_new_lines_source_code(sc_df)
    sc_df = clean_composite_source_code(sc_df)
    sc_df = clean_filepath(sc_df)
    return sc_df

def clean_bug_df(sc_df):
    sc_df = clean_new_lines_bug_report(sc_df)
    sc_df = clean_composite_bug_report(sc_df)
    return sc_df

# get clean versions of the dataframes
sc_df = clean_source_df(all_projects_source_codes)
br_df = clean_bug_df(all_projects_bugreports)

# TODO figure out which we need from the bug report 
#           do we use summary or description or both?
# bug_df = clean_source_code_list(all_projects_bugreports)


# get a list of the projects and their parent project
print(sc_df.project.unique())
projects = sc_df.project.unique()


# group the data frames
grouped_df = sc_df.groupby(sc_df.project)

# example of getting a single data frame
col_df = grouped_df.get_group("COLLECTIONS")
display(col_df)

# Won't use any of this
# commons_projects = projects[0:4]
# spring_projects = projects[4:11]
# wildfly_projects = projects[11]

# config_df = grouped_df.get_group(commons_projects[1])
# ely_df = grouped_df.get_group(wildfly_projects)
# ldap_df = grouped_df.get_group(spring_projects[3])

# print(config_df.iloc[0].filename)
# print(col_df.iloc[0].filename)
# print(ely_df.iloc[0].filename)
# print(ldap_df.iloc[0].filename)


['COLLECTIONS' 'CONFIGURATION' 'IO' 'LANG' 'DATACMNS' 'DATAMONGO'
 'DATAREST' 'LDAP' 'SEC' 'SOCIALFB' 'SPR' 'ELY']


,filename,unprocessed_code,project
0,org.apache.commons.collections.arraystack.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
1,org.apache.commons.collections.bag.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
2,org.apache.commons.collections.bagutils.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
3,org.apache.commons.collections.beanmap.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
4,org.apache.commons.collections.bidimap.java,Licensed to the Apache Software Foundation ...,COLLECTIONS
...,...,...,...
471,org.apache.commons.collections.set.testtransfo...,Licensed to the Apache Software Foundation ...,COLLECTIONS
472,org.apache.commons.collections.set.testtypedse...,Licensed to the Apache Software Foundation ...,COLLECTIONS
473,org.apache.commons.collections.set.testtypedso...,Licensed to the Apache Software Foundation ...,COLLECTIONS
474,org.apache.commons.collections.set.testunmodif...,Licensed to the Apache Software Foundation ...,COLLECTIONS


In [29]:

def gen_parent_df(group, names):
    result = pd.DataFrame
    temp = []
    for name in names:
        temp.append(group.get_group(name))
    
    result = pd.concate(temp, ignore_index = true)    
    return result

# don't know if we are looking at these groups OR each of the projects individually
# commons_df = gen_parent_df(grouped_df, commons_projects)
# sprint_df = gen_parent_df(grouped_df, spring_projects)
# wildfly_df = gen_parent_df(grouped_df, wildfly_projects)




In [11]:
from sklearn.metrics.pairwise import cosine_similarity


def calculate_similarity(source_code, bug):
    cosSim = cosine_similarity(bug, source_code).flatten()
    return cosSim

# fit the vectorizer and transform data
def transform_data(source_code_data, query_data):
    # DO WE SET min_df?  default normalization is l2
    vect = TfidfVectorizer(min_df=1).fit(source_code_data)
    X = vect.transform(source_code_data)
    y = vect.transform(query_data)
    similarity = calculate_similarity(X,y)
    return X, y, similarity

# remove the stem and stop words
def stem_stop(text):
    stemmer = PorterStemmer()   #"english"
    text = text.split()
    text = [w for w in text if not w in STOP_WORDS]
    text = list(map(lambda x: stemmer.stem(x), text))
    text = ' '.join(text)
    text = text.strip()

    return text

# first stem and remove stop words from the data frames column
# example 

# source_code_df['unprocessed_code'] = source_code_df['unprocessed_code'].map(stem_stop)
# bug_df['unprocessed_code'] = bug_df['unprocessed_code'].map(stem_stop)

# then get the dataframe column you want to pass as source_code_data and as the query_data(bug)




In [ ]:
# similarity vector should be a data frame with bug_report, bug_report_fix, source_code.filename, and the similarity score

# another function that determines if the source_code file name is in the bugreport fix array. 

